# Segment clouds and cloud shadows in Sentinel-2 images (L1C)
This notebook shows an example on how to use [ukis-csmask](https://github.com/dlr-eoc/ukis-csmask) to segment clouds and cloud shadows in Level-1C images from Sentinel-2 using only 4 spectral bands. Images are loaded from local file system. Here we use [ukis-pysat](https://github.com/dlr-eoc/ukis-pysat) for convencience image handling, but you can also work directly with [numpy](https://numpy.org/) arrays.

> NOTE: to run this notebook, we first need to install some additional dependencies for image handling
```shell
pip install ukis-pysat[complete]
```

In [ ]:
from pathlib import Path
from ukis_csmask.mask import CSmask
from ukis_pysat.raster import Image, Platform

In [ ]:
# user settings
img_file = "sentinel2.tif"
product_level = "l1c"
band_order = ["blue", "green", "red", "nir"]
providers = ["CUDAExecutionProvider"]
out_dir = "ukis-csmask/examples"

In [ ]:
# read Level-1C image from file, convert digital numbers to TOA reflectance
# and make sure resolution is 30 m to get best performance
img = Image(data=img_file, dimorder="last")
img.dn2toa(platform=Platform.Sentinel2, wavelengths=band_order)
img.warp(resampling_method=0, resolution=30, dst_crs=img.dataset.crs)

In [ ]:
# compute cloud and cloud shadow mask
csmask = CSmask(
    img=img.arr,
    band_order=band_order,
    product_level=product_level,
    nodata_value=0,
    invalid_buffer=4,
    intra_op_num_threads=0,
    inter_op_num_threads=0,
    providers=providers,
    batch_size=1,
)

# access cloud and cloud shadow mask as numpy array
csm = csmask.csm

# access valid mask as numpy array
valid = csmask.valid

In [ ]:
# convert results to ukis-pysat Image
# this assigns back the georeference
csm = Image(csm, transform=img.dataset.transform, crs=img.dataset.crs, dimorder="last")
valid = Image(valid, transform=img.dataset.transform, crs=img.dataset.crs, dimorder="last")

# write results to file
csm.write_to_file(
    path_to_file=Path(out_dir) / Path(f"{Path(img_file).name}_csm.tif"),
    dtype=csm.dtype,
    driver="COG",
    compress="LZW",
    kwargs={"BLOCKSIZE": 512, "BIGTIFF": "IF_SAFER"},
)
valid.write_to_file(
    path_to_file=Path(out_dir) / Path(f"{Path(img_file).name}_valid.tif"),
    dtype=valid.dtype,
    driver="COG",
    compress="LZW",
    kwargs={"BLOCKSIZE": 512, "BIGTIFF": "IF_SAFER"},
)